In [1]:
import gym

from copy import deepcopy
print(gym.__path__)
env = gym.make('Ant-v3')
o = env.reset()
old_state = deepcopy(env.sim.get_state())

def setEnvWithState(env, state):
    env.sim.set_state(state)
    env.sim.forward()
    return env.get_obs()

def setEnvWithObs(env, obs):
    state = deepcopy(env.sim.get_state())
    state.qpos = obs[:state.qpos.shape[0]]
    state.qvel = obs[state.qpos.shape[0]:state.qpos.shape[0]+state.qvel.shape[0]]
    env.sim.set_state(state)
    env.sim.forward()
    return env.get_obs()


print(o.shape)
print(o[:old_state.qpos.shape[0]].shape)
print(o[old_state.qpos.shape[0]:old_state.qpos.shape[0]+old_state.qvel.shape[0]].shape)
print(old_state.qpos.flat.copy().shape)
print(old_state.qvel.flat.copy().shape)
print(env.get_obs().shape)


['/home/lclan/anaconda3/envs/rob_sac/lib/python3.7/site-packages/gym']
(111,)
(15,)
(14,)
(15,)
(14,)
(111,)


/home/lclan/anaconda3/envs/rob_sac/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [9]:
import numpy as np
from scipy import stats
import os
import torch
import os.path as osp
def test_model(env, model, max_ep_len=None, num_episodes=20, interval = 200):
    o, r, d, ep_ret, ep_len, n = env.reset(), 0, False, 0, 0, 0
    total_rewards = []
    trajs = []
    mid_points = []
    while n < num_episodes:
        old_state = deepcopy(env.sim.get_state())
        mid_points.append((ep_len, old_state, o))
        a = get_action(o, model)
        o, r, d, _ = env.step(a)
        ep_ret += r
        ep_len += 1
        if d or (ep_len == max_ep_len):
            total_rewards.append(ep_ret)
            trajs.append(mid_points)
            mid_points = [] # clear mid point
            o, r, d, ep_ret, ep_len = env.reset(), 0, False, 0, 0
            n += 1
    return total_rewards, trajs

def get_models(path, env_name, name):
    env = gym.make(env_name)
    fpath = osp.join(path, name)
    print(fpath)
    models = []
    while 1:
        fname = osp.join(fpath, name + "_s" + str(len(models)) ,'pyt_save', 'model.pt')
        print(fname)
        if(osp.exists(fname)):
            model = torch.load(fname)
            models.append(model)
        else:
            break
    # print(len(models))
    test_val = []
    env = gym.make(env_name)
    for model in models:
        x, _ = test_model(env, model, num_episodes=5)
        # print(x, stats.trim_mean(x, 0.1))
        test_val.append(stats.trim_mean(x, 0.1))
    sorted_ids = np.argsort(test_val)
    model1 = models[sorted_ids[-1]]
    model2 = models[sorted_ids[-2]]
    return model1, model2

def get_action(o, md):
    with torch.no_grad():
        o = torch.as_tensor(o, dtype=torch.float32)
        a = md.act(o)
    return a

In [10]:
m1, m2 = get_models('/home/lclan/spinningup/data/', 'Ant-v3', 'Ant-v3_sac_base')


/home/lclan/anaconda3/envs/rob_sac/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


/home/lclan/spinningup/data/Ant-v3_sac_base
/home/lclan/spinningup/data/Ant-v3_sac_base/Ant-v3_sac_base_s0/pyt_save/model.pt
/home/lclan/spinningup/data/Ant-v3_sac_base/Ant-v3_sac_base_s1/pyt_save/model.pt
/home/lclan/spinningup/data/Ant-v3_sac_base/Ant-v3_sac_base_s2/pyt_save/model.pt
/home/lclan/spinningup/data/Ant-v3_sac_base/Ant-v3_sac_base_s3/pyt_save/model.pt
/home/lclan/spinningup/data/Ant-v3_sac_base/Ant-v3_sac_base_s4/pyt_save/model.pt


In [72]:
def get_value(o, md):
    # with torch.no_grad():
    a, _ = md.pi(o, deterministic=True, with_logprob=False)
    q1 = md.q1(o, a)
    q2 = md.q2(o, a)
    q = torch.min(q1, q2)
    return q

def set_state_with_obs(env, obs):
    if(torch.is_tensor(obs)):
        obs = obs.detach().numpy()
    nq = env.model.nq
    nv = env.model.nv
    qpos = obs[:nq]
    qvel = obs[nq:nq+nv]
    env.set_state(qpos, qvel)
    new_obs = env.get_obs()
    return new_obs

def save_state(env):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    return (qpos ,qvel)

def restore_state(env, old_state):
    env.set_state(old_state[0], old_state[1])
    new_obs = env.get_obs()
    return new_obs

max_eps_t = 0.0001
env = gym.make('Ant-v3')
o = env.reset()
old_state = save_state(env)

with torch.torch.no_grad():
    oo2 = torch.as_tensor(o, dtype=torch.float32)
    q2 = get_value(oo2, m1)
    print(q2.detach().numpy())


with torch.set_grad_enabled(True):
    oo = torch.as_tensor(o, dtype=torch.float32)
    print(oo.requires_grad)
    oo.requires_grad = True
    q = get_value(oo, m1)
    grad_o = torch.autograd.grad(q, [oo], retain_graph=True)[0]


o_adv = oo - max_eps_t * grad_o.detach().sign()

with torch.torch.no_grad():
    oo2 = torch.as_tensor(o_adv, dtype=torch.float32)
    q2 = get_value(oo2, m1)
    print(q2.detach().numpy())

new_obs = set_state_with_obs(env, o_adv)

with torch.torch.no_grad():
    oo2 = torch.as_tensor(new_obs, dtype=torch.float32)
    q2 = get_value(oo2, m1)
    print(q2.detach().numpy())



    



366.19022
False
366.025
-19.158342


/home/lclan/anaconda3/envs/rob_sac/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [73]:
def run_extra_steps(env, o, ep_len, md, max_ep_len, step_num = 50):
    # return 0
    # print(env.done)
    total_r = 0
    for i in range(step_num):
        a = get_action(o, md)
        o, r, d, _ = env.step(a)
        total_r += r
        ep_len += 1
        if d or (ep_len == max_ep_len):
            # print(i, d, r, ep_len, max_ep_len)
            return (d, total_r)
    return (d, total_r)

d, r = run_extra_steps(env, new_obs, 0, m1, 1000, 200)
print(r)
new_obs = restore_state(env, old_state)
d, r = run_extra_steps(env, new_obs, 0, m1, 1000, 200)
print(r)


2.192668482775532
1155.4254482090028


In [125]:
def get_value(o, md):
    if not torch.is_tensor(o):
        o = torch.as_tensor(o, dtype=torch.float32)
    with torch.no_grad():
        a, _ = md.pi(o, deterministic=True, with_logprob=False)
        q1 = md.q1(o, a)
        q2 = md.q2(o, a)
        q = torch.min(q1, q2)
    return q

def get_grad(env, md, eps=0.001):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    o = env.get_obs()
    old_q = get_value(o, md).numpy()
    grad_pos = np.zeros_like(qpos)
    for i in range(len(qpos)):
        qpos[i]+=eps
        env.set_state(qpos, qvel)
        o = env.get_obs()
        q = get_value(o, md).numpy()
        grad_pos[i] = (q - old_q) / eps        
        qpos[i]-=eps
        
    grad_vel = np.zeros_like(qvel)
    for i in range(len(qvel)):
        qvel[i]+=eps
        env.set_state(qpos, qvel)
        o = env.get_obs()
        q = get_value(o, md).numpy()
        grad_vel[i] = (q - old_q) / eps        
        qvel[i]-=eps
    return grad_pos, grad_vel

def change_env_to_adv(env, md, max_eps_t):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    grad_pos, grad_vel = get_grad(env, md, max_eps_t)
    # grad_pos = np.clip(grad_pos, -1, 1)
    # grad_vel = np.clip(grad_vel, -1, 1)
    adv_qpos = qpos - max_eps_t*grad_pos
    adv_qvel = qvel - max_eps_t*grad_vel
    env.set_state(adv_qpos, adv_qvel)
    new_obs = env.get_obs()
    return new_obs

max_eps_t = 0.01
env = gym.make('Ant-v3')
o = env.reset()
old_state = save_state(env)
with torch.torch.no_grad():
    oo2 = torch.as_tensor(o, dtype=torch.float32)
    q2 = get_value(oo2, m1)
    print(q2.detach().numpy())

o = change_env_to_adv(env, m1, max_eps_t)
with torch.torch.no_grad():
    oo2 = torch.as_tensor(o, dtype=torch.float32)
    q2 = get_value(oo2, m1)
    print(q2.detach().numpy())

383.5152
-8.68798


In [130]:
import spinup.algos.pytorch.sac.core as core
from torch.optim import Adam
import itertools
class ReplayBuffer:
    """
    A simple FIFO experience replay buffer for SAC agents.
    """

    def __init__(self, obs_dim, act_dim, size):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.obs2_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.ptr, self.size, self.max_size = 0, 0, size

    def store(self, obs, act, rew, next_obs, done):
        self.obs_buf[self.ptr] = obs
        self.obs2_buf[self.ptr] = next_obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def sample_batch(self, batch_size=32):
        idxs = np.random.randint(0, self.size, size=batch_size)
        batch = dict(obs=self.obs_buf[idxs],
                     obs2=self.obs2_buf[idxs],
                     act=self.act_buf[idxs],
                     rew=self.rew_buf[idxs],
                     done=self.done_buf[idxs])
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in batch.items()}

In [131]:
def get_action(ac, o, deterministic=False):
    return ac.act(torch.as_tensor(o, dtype=torch.float32), 
                    deterministic)
def get_replay_buff(ac, replay_size = int(1e6)):
    obs_dim = env.observation_space.shape
    act_dim = env.action_space.shape[0]
    replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=replay_size)
    o, ep_ret, ep_len = env.reset(), 0, 0
    max_ep_len = 100
    rs = []
    for t in range(replay_size):
        if t % 100000 == 0:
            print(t)
        a = get_action(ac, o, deterministic=False)
        o2, r, d, _ = env.step(a)
        ep_ret += r
        ep_len += 1
        d = False if ep_len==max_ep_len else d
        replay_buffer.store(o, a, r, o2, d)
        o = o2
        if d or (ep_len == max_ep_len):
            rs.append(ep_ret)
            o, ep_ret, ep_len = env.reset(), 0, 0
    
    return replay_buffer

replay_buffer = get_replay_buff(m1)
    

0
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [133]:
def compute_loss_q(ac, ac_targ, data):
    gamma = 0.99
    o, a, r, o2, d = data['obs'], data['act'], data['rew'], data['obs2'], data['done']
    q1 = ac.q1(o,a)
    q2 = ac.q2(o,a)
    with torch.no_grad():
        a2, _ = ac.pi(o2)
        q1_pi_targ = ac_targ.q1(o2, a2)
        q2_pi_targ = ac_targ.q2(o2, a2)
        q_pi_targ = torch.min(q1_pi_targ, q2_pi_targ)
        backup = r + gamma * (1 - d) * (q_pi_targ)
    loss_q1 = ((q1 - backup)**2).mean()
    loss_q2 = ((q2 - backup)**2).mean()
    loss_q = loss_q1 + loss_q2
    return loss_q

def train_with_buffer(ac, replay_buffer):
    ac_targ = deepcopy(ac)
    polyak=0.995
    for p in ac_targ.parameters():
        p.requires_grad = False
    q_params = itertools.chain(ac.q1.parameters(), ac.q2.parameters())
    q_optimizer = Adam(q_params, lr=1e-3)
    batch_size = 32
    total_steps = (int)(replay_buffer.max_size / batch_size)
    for t in range(total_steps):
        batch = replay_buffer.sample_batch(batch_size)
        q_optimizer.zero_grad()
        loss_q = compute_loss_q(ac, ac_targ, batch)
        loss_q.backward()
        q_optimizer.step()
        with torch.no_grad():
            for p, p_targ in zip(ac.parameters(), ac_targ.parameters()):
                # NB: We use an in-place operations "mul_", "add_" to update target
                # params, as opposed to "mul" and "add", which would make new tensors.
                p_targ.data.mul_(polyak)
                p_targ.data.add_((1 - polyak) * p.data)

md = deepcopy(m1)
train_with_buffer(md, replay_buffer)


In [147]:
def print_2f(*args):
    __builtins__.print(*("%.2f" % a if isinstance(a, float) else a
                         for a in args))

def get_action(o, md):
    with torch.no_grad():
        o = torch.as_tensor(o, dtype=torch.float32)
        a = md.act(o)
    return a

def run_extra_steps(env, o, ep_len, md, max_ep_len, step_num = 50):
    total_r = 0
    for i in range(step_num):
        a = get_action(o, md)
        o, r, d, _ = env.step(a)
        total_r += r
        ep_len += 1
        if d or (ep_len == max_ep_len):
            # print(i, d, r, ep_len, max_ep_len)
            return (d, total_r)
    return (d, total_r)

def get_value(o, md):
    if not torch.is_tensor(o):
        o = torch.as_tensor(o, dtype=torch.float32)
    with torch.no_grad():
        a, _ = md.pi(o, deterministic=True, with_logprob=False)
        q1 = md.q1(o, a)
        q2 = md.q2(o, a)
        q = torch.min(q1, q2)
    return q

def get_grad(env, md, eps=0.01):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    o = env.get_obs()
    old_q = get_value(o, md).numpy()
    grad_pos = np.zeros_like(qpos)
    for i in range(len(qpos)):
        qpos[i]+=eps
        env.set_state(qpos, qvel)
        o = env.get_obs()
        q = get_value(o, md).numpy()
        grad_pos[i] = (q - old_q) / eps        
        qpos[i]-=eps
        
    grad_vel = np.zeros_like(qvel)
    for i in range(len(qvel)):
        qvel[i]+=eps
        env.set_state(qpos, qvel)
        o = env.get_obs()
        q = get_value(o, md).numpy()
        grad_vel[i] = (q - old_q) / eps        
        qvel[i]-=eps
    return grad_pos, grad_vel

def change_env_to_adv(env, md, max_eps_t):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    grad_pos, grad_vel = get_grad(env, md, max_eps_t*0.1)
    grad_pos = np.clip(grad_pos, -1, 1)
    grad_vel = np.clip(grad_vel, -1, 1)
    # adv_qpos = qpos + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nq)
    # adv_qvel = qvel + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nv)
    adv_qpos = qpos - max_eps_t*grad_pos
    adv_qvel = qvel - max_eps_t*grad_vel
    init_qpos = env.sim.data.qpos.ravel().copy()
    init_qvel = env.sim.data.qvel.ravel().copy()
    # rns = env._reset_noise_scale
    rns = 0.1
    adv_qpos = np.clip(adv_qpos, init_qpos - rns,  init_qpos + rns)
    adv_qvel = np.clip(adv_qvel, init_qvel - 2*rns, init_qvel + 2*rns)
    
    env.set_state(adv_qpos, adv_qvel)
    new_obs = env.get_obs()
    return new_obs

def save_state(env):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    return (qpos ,qvel)

def restore_state(env, old_state):
    env.reset()
    env.set_state(old_state[0], old_state[1])
    new_obs = env.get_obs()
    return new_obs

def get_mean(v):
    return sum(v)/len(v)

def testing(md, env_name, max_eps_t = 0.01):
    test_num = 50
    test_l = 1000
    env = gym.make(env_name)
    orig_rs = []
    adv_rs = []
    orig_qs = []
    adv_qs = []
    for i in range(test_num):
        o = env.reset()
        old_state = save_state(env)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        orig_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        orig_rs.append(r)
        
        restore_state(env, old_state)
        for _ in range(20):
            o = change_env_to_adv(env, md, max_eps_t)

        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        adv_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        adv_rs.append(r)
    print_2f(get_mean(orig_rs), get_mean(adv_rs), get_mean(orig_qs), get_mean(adv_qs))

testing(m1, 'Ant-v3')
testing(md, 'Ant-v3')

5700.78 5146.02 379.25 224.51
5644.68 3912.45 368.31 3.62


In [151]:
def get_action(ac, o, deterministic=False):
    return ac.act(torch.as_tensor(o, dtype=torch.float32), 
                    deterministic)
def get_adv_replay_buff(ac, replay_size = int(1e6)):
    obs_dim = env.observation_space.shape
    act_dim = env.action_space.shape[0]
    replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=replay_size)
    o, ep_ret, ep_len = env.reset(), 0, 0
    max_ep_len = 1000
    rs = []
    use_adv = False
    for t in range(replay_size):
        if t % 100000 == 0:
            print(t)
        a = get_action(ac, o, deterministic=False)
        o2, r, d, _ = env.step(a)
        ep_ret += r
        ep_len += 1
        d = False if ep_len==max_ep_len else d
        replay_buffer.store(o, a, r, o2, d)
        o = o2
        if d or (ep_len == max_ep_len):
            rs.append(ep_ret)
            o, ep_ret, ep_len = env.reset(), 0, 0
            if(use_adv):
                use_adv = False
                for _ in range(10):
                    o = change_env_to_adv(env, md, max_eps_t)
            else:
                use_adv = True
    
    return replay_buffer

In [152]:
replay_buffer2 =  get_adv_replay_buff(md)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [153]:
def compute_loss_q(ac, ac_targ, data):
    gamma = 0.99
    o, a, r, o2, d = data['obs'], data['act'], data['rew'], data['obs2'], data['done']
    q1 = ac.q1(o,a)
    q2 = ac.q2(o,a)
    with torch.no_grad():
        a2, _ = ac.pi(o2)
        q1_pi_targ = ac_targ.q1(o2, a2)
        q2_pi_targ = ac_targ.q2(o2, a2)
        q_pi_targ = torch.min(q1_pi_targ, q2_pi_targ)
        backup = r + gamma * (1 - d) * (q_pi_targ)
    loss_q1 = ((q1 - backup)**2).mean()
    loss_q2 = ((q2 - backup)**2).mean()
    loss_q = loss_q1 + loss_q2
    return loss_q

def train_with_buffer(ac, replay_buffer):
    ac_targ = deepcopy(ac)
    polyak=0.995
    for p in ac_targ.parameters():
        p.requires_grad = False
    q_params = itertools.chain(ac.q1.parameters(), ac.q2.parameters())
    q_optimizer = Adam(q_params, lr=1e-3)
    batch_size = 32
    total_steps = (int)(replay_buffer.max_size / batch_size)
    for t in range(total_steps):
        batch = replay_buffer.sample_batch(batch_size)
        q_optimizer.zero_grad()
        loss_q = compute_loss_q(ac, ac_targ, batch)
        loss_q.backward()
        q_optimizer.step()
        with torch.no_grad():
            for p, p_targ in zip(ac.parameters(), ac_targ.parameters()):
                # NB: We use an in-place operations "mul_", "add_" to update target
                # params, as opposed to "mul" and "add", which would make new tensors.
                p_targ.data.mul_(polyak)
                p_targ.data.add_((1 - polyak) * p.data)

md2 = deepcopy(md)
train_with_buffer(md2, replay_buffer2)

In [155]:
def get_action(o, md):
    with torch.no_grad():
        o = torch.as_tensor(o, dtype=torch.float32)
        a = md.act(o)
    return a

def testing(md, env_name, adv_step, max_eps_t = 0.01):
    test_num = 50
    test_l = 1000
    env = gym.make(env_name)
    orig_rs = []
    adv_rs = []
    orig_qs = []
    adv_qs = []
    for i in range(test_num):
        o = env.reset()
        old_state = save_state(env)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        orig_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        orig_rs.append(r)
        
        restore_state(env, old_state)
        for _ in range(adv_step):
            o = change_env_to_adv(env, md, max_eps_t)

        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        adv_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        adv_rs.append(r)
    print_2f(adv_step, get_mean(orig_rs), get_mean(adv_rs), get_mean(orig_qs), get_mean(adv_qs))

testing(m1, 'Ant-v3', 10)
testing(md, 'Ant-v3', 10)
testing(md2, 'Ant-v3', 10)
testing(m1, 'Ant-v3', 20)
testing(md, 'Ant-v3', 20)
testing(md2, 'Ant-v3', 20)
testing(m1, 'Ant-v3', 40)
testing(md, 'Ant-v3', 40)
testing(md2, 'Ant-v3', 40)
testing(m1, 'Ant-v3', 80)
testing(md, 'Ant-v3', 80)
testing(md2, 'Ant-v3', 80)

/home/lclan/anaconda3/envs/rob_sac/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


10 5706.19 5588.91 373.61 316.58
10 5813.85 5820.34 367.53 263.14
10 5683.75 5962.37 368.95 301.56
20 5849.45 4411.02 376.55 187.66
20 5773.06 4970.68 366.53 36.66
20 5663.79 2920.11 366.33 41.21
40 5568.68 2379.38 373.70 -29.58
40 5872.23 936.78 367.03 -227.79
40 5860.24 313.66 368.74 -317.25
80 5830.76 458.36 376.87 -418.50
80 5665.79 -11.66 369.78 -486.98
80 5825.31 -5.85 367.49 -712.43


In [175]:
def get_action(o, md):
    with torch.no_grad():
        o = torch.as_tensor(o, dtype=torch.float32)
        a = md.act(o)
    return a

def get_state_dis(s1, s2):
    l = len(s1[0]) + len(s1[1])
    l1_mean_dis = (sum(np.abs(s1[0]-s2[0])) + sum(np.abs(s1[1]-s2[1]))) / l
    l2_mean_dis = np.sqrt((sum((s1[0]-s2[0])**2) + sum((s1[1]-s2[1])**2))/l)
    return l1_mean_dis, l2_mean_dis
        
def get_init_state(env):
    env.sim.reset()
    init_qpos = env.sim.data.qpos.ravel().copy()
    init_qvel = env.sim.data.qvel.ravel().copy()
    return (init_qpos, init_qvel)
def print_2f(*args):
    __builtins__.print(*("%.2f" % a if isinstance(a, float) else a
                         for a in args), end=' ')

In [169]:
a = np.array([1,3,2])
b = np.array([2,4,1])
print(sum(np.abs(a-b)))
print(np.sqrt(sum((a-b)**2)/3))


3
1.0


In [178]:

def change_env_to_adv(env, md, max_eps_t, init_state):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    grad_pos, grad_vel = get_grad(env, md, max_eps_t)
    grad_pos = np.sign(grad_pos)
    grad_vel = np.sign(grad_vel)
    # grad_pos = np.clip(grad_pos, -1, 1)
    # grad_vel = np.clip(grad_vel, -1, 1)
    # adv_qpos = qpos + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nq)
    # adv_qvel = qvel + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nv)
    adv_qpos = qpos - max_eps_t*grad_pos
    adv_qvel = qvel - 2.0* max_eps_t*grad_vel
    init_qpos = init_state[0]
    init_qvel = init_state[1]
    # rns = env._reset_noise_scale
    rns = 0.1
    adv_qpos = np.clip(adv_qpos, init_qpos - rns,  init_qpos + rns)
    adv_qvel = np.clip(adv_qvel, init_qvel - 2*rns, init_qvel + 2*rns)
    
    env.set_state(adv_qpos, adv_qvel)
    new_obs = env.get_obs()
    return new_obs

def testing(md, env_name, adv_step, max_eps_t = 0.1):
    test_num = 50
    test_l = 1000
    env = gym.make(env_name)
    orig_rs = []
    adv_rs = []
    orig_qs = []
    adv_qs = []
    adv_step_size = 1.5 * max_eps_t / adv_step
    l1_old, l2_old, l1_init, l2_init = [], [], [], []
    init_state = get_init_state(env)
    for i in range(test_num):
        o = env.reset()
        old_state = save_state(env)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        orig_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        orig_rs.append(r)
        
        restore_state(env, old_state)
        for _ in range(adv_step):
            o = change_env_to_adv(env, md, adv_step_size, init_state)
        new_state = save_state(env)
        l1, l2 = get_state_dis(new_state, old_state)
        l1_old.append(l1)
        l2_old.append(l2)
        l1, l2 = get_state_dis(new_state, init_state)
        l1_init.append(l1)
        l2_init.append(l2)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        adv_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        adv_rs.append(r)
    print_2f(adv_step, adv_step_size, get_mean(orig_rs), get_mean(adv_rs), get_mean(orig_qs), get_mean(adv_qs))
    print_2f("dis: ", get_mean(l1_old), get_mean(l2_old), get_mean(l1_init), get_mean(l2_init))
    

testing(m1, 'Ant-v3', 10)
testing(md, 'Ant-v3', 10)
testing(md2, 'Ant-v3', 10)
testing(m1, 'Ant-v3', 20)
testing(md, 'Ant-v3', 20)
testing(md2, 'Ant-v3', 20)
testing(m1, 'Ant-v3', 40)
testing(md, 'Ant-v3', 40)
testing(md2, 'Ant-v3', 40)
testing(m1, 'Ant-v3', 80)
testing(md, 'Ant-v3', 80)
testing(md2, 'Ant-v3', 80)


/home/lclan/anaconda3/envs/rob_sac/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


10 5721.09 5872.24 376.68 317.13
dis:  0.11 0.14 0.11 0.13
10 5973.47 5852.39 369.21 280.29
dis:  0.11 0.14 0.12 0.14
10 5892.71 5661.84 367.57 307.97
dis:  0.11 0.14 0.12 0.14
20 5733.54 5672.60 374.34 314.58
dis:  0.11 0.14 0.12 0.13
20 5750.34 5803.90 369.24 278.52
dis:  0.11 0.15 0.12 0.14
20 5922.85 5745.58 365.62 306.31
dis:  0.11 0.14 0.12 0.14
40 5560.87 5590.67 377.79 316.40
dis:  0.11 0.14 0.12 0.13
40 5581.64 5463.63 366.13 278.84
dis:  0.11 0.14 0.12 0.14
40 5677.23 5723.21 366.03 308.41
dis:  0.11 0.14 0.12 0.14
80 5806.91 5795.64 376.87 313.66
dis:  0.11 0.14 0.12 0.13
80 5889.10 5823.45 365.69 280.15
dis:  0.11 0.14 0.12 0.14
80 5838.21 5731.73 368.39 309.61
dis:  0.11 0.14 0.12 0.14


In [180]:
def change_env_to_adv(env, md, max_eps_t, init_state):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    grad_pos, grad_vel = get_grad(env, md, max_eps_t)
    grad_pos = np.clip(grad_pos, -1, 1)
    grad_vel = np.clip(grad_vel, -1, 1)
    # adv_qpos = qpos + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nq)
    # adv_qvel = qvel + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nv)
    adv_qpos = qpos - max_eps_t*grad_pos
    adv_qvel = qvel - max_eps_t*grad_vel
    init_qpos = init_state[0]
    init_qvel = init_state[1]
    # rns = env._reset_noise_scale
    rns = 0.1
    adv_qpos = np.clip(adv_qpos, init_qpos - rns,  init_qpos + rns)
    adv_qvel = np.clip(adv_qvel, init_qvel - 2*rns, init_qvel + 2*rns)
    
    env.set_state(adv_qpos, adv_qvel)
    new_obs = env.get_obs()
    return new_obs

def testing(md, env_name, adv_step, max_eps_t = 0.001):
    test_num = 50
    test_l = 1000
    env = gym.make(env_name)
    orig_rs = []
    adv_rs = []
    orig_qs = []
    adv_qs = []
    l1_old, l2_old, l1_init, l2_init = [], [], [], []
    init_state = get_init_state(env)
    for i in range(test_num):
        o = env.reset()
        old_state = save_state(env)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        orig_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        orig_rs.append(r)
        
        restore_state(env, old_state)
        for _ in range(adv_step):
            o = change_env_to_adv(env, md, max_eps_t, init_state)
        new_state = save_state(env)
        l1, l2 = get_state_dis(new_state, old_state)
        l1_old.append(l1)
        l2_old.append(l2)
        l1, l2 = get_state_dis(new_state, init_state)
        l1_init.append(l1)
        l2_init.append(l2)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        adv_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        adv_rs.append(r)
    print_2f(adv_step, get_mean(orig_rs), get_mean(adv_rs), get_mean(orig_qs), get_mean(adv_qs), "dis: ", get_mean(l1_old), get_mean(l2_old), get_mean(l1_init), get_mean(l2_init)) 

testing(m1, 'Ant-v3', 10)
testing(md, 'Ant-v3', 10)
testing(md2, 'Ant-v3', 10)
testing(m1, 'Ant-v3', 20)
testing(md, 'Ant-v3', 20)
testing(md2, 'Ant-v3', 20)
testing(m1, 'Ant-v3', 40)
testing(md, 'Ant-v3', 40)
testing(md2, 'Ant-v3', 40)
testing(m1, 'Ant-v3', 80)
testing(md, 'Ant-v3', 80)
testing(md2, 'Ant-v3', 80)


/home/lclan/anaconda3/envs/rob_sac/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


10 5741.57 5819.51 376.24 371.54 dis:  0.01 0.01 0.06 0.08
10 5730.79 5671.04 366.41 360.79 dis:  0.01 0.01 0.06 0.07
10 5763.54 5307.02 368.64 363.66 dis:  0.01 0.01 0.06 0.07
20 5962.47 5620.17 378.11 369.14 dis:  0.02 0.02 0.07 0.08
20 5583.43 5698.25 368.21 357.66 dis:  0.02 0.02 0.06 0.08
20 5596.06 5479.15 369.01 359.34 dis:  0.02 0.02 0.06 0.08
40 5625.67 5758.66 377.52 359.95 dis:  0.03 0.03 0.07 0.08
40 5675.47 5865.46 368.83 345.89 dis:  0.03 0.03 0.07 0.08
40 5571.88 5956.84 368.87 351.24 dis:  0.03 0.03 0.07 0.08
80 5796.22 5876.63 376.36 342.60 dis:  0.05 0.06 0.08 0.09
80 5770.99 5894.51 366.82 325.21 dis:  0.05 0.06 0.08 0.09
80 5706.64 5815.35 366.58 334.24 dis:  0.05 0.06 0.08 0.10


In [181]:
def change_env_to_adv(env, md, max_eps_t, init_state):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    grad_pos, grad_vel = get_grad(env, md, max_eps_t*0.1)
    grad_pos = np.clip(grad_pos, -1, 1)
    grad_vel = np.clip(grad_vel, -1, 1)
    # adv_qpos = qpos + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nq)
    # adv_qvel = qvel + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nv)
    adv_qpos = qpos - max_eps_t*grad_pos
    adv_qvel = qvel - max_eps_t*grad_vel
    init_qpos = init_state[0]
    init_qvel = init_state[1]
    # rns = env._reset_noise_scale
    rns = 0.1
    # adv_qpos = np.clip(adv_qpos, init_qpos - rns,  init_qpos + rns)
    # adv_qvel = np.clip(adv_qvel, init_qvel - 2*rns, init_qvel + 2*rns)
    
    env.set_state(adv_qpos, adv_qvel)
    new_obs = env.get_obs()
    return new_obs

def testing(md, env_name, adv_step, max_eps_t = 0.01):
    test_num = 50
    test_l = 1000
    env = gym.make(env_name)
    orig_rs = []
    adv_rs = []
    orig_qs = []
    adv_qs = []
    l1_old, l2_old, l1_init, l2_init = [], [], [], []
    init_state = get_init_state(env)
    for i in range(test_num):
        o = env.reset()
        old_state = save_state(env)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        orig_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        orig_rs.append(r)
        
        restore_state(env, old_state)
        for _ in range(adv_step):
            o = change_env_to_adv(env, md, max_eps_t, init_state)
        new_state = save_state(env)
        l1, l2 = get_state_dis(new_state, old_state)
        l1_old.append(l1)
        l2_old.append(l2)
        l1, l2 = get_state_dis(new_state, init_state)
        l1_init.append(l1)
        l2_init.append(l2)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        adv_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        adv_rs.append(r)
    print_2f(adv_step, get_mean(orig_rs), get_mean(adv_rs), get_mean(orig_qs), get_mean(adv_qs), "dis: ", get_mean(l1_old), get_mean(l2_old), get_mean(l1_init), get_mean(l2_init)) 

testing(m1, 'Ant-v3', 10)
testing(md, 'Ant-v3', 10)
testing(md2, 'Ant-v3', 10)
testing(m1, 'Ant-v3', 20)
testing(md, 'Ant-v3', 20)
testing(md2, 'Ant-v3', 20)
testing(m1, 'Ant-v3', 40)
testing(md, 'Ant-v3', 40)
testing(md2, 'Ant-v3', 40)
testing(m1, 'Ant-v3', 80)
testing(md, 'Ant-v3', 80)
testing(md2, 'Ant-v3', 80)


/home/lclan/anaconda3/envs/rob_sac/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


10 5786.07 5772.12 377.05 318.39 dis:  0.07 0.08 0.09 0.11
10 5859.89 5729.60 369.58 281.53 dis:  0.07 0.08 0.09 0.11
10 5716.49 5603.95 366.56 290.38 dis:  0.07 0.08 0.09 0.11
20 5900.58 4438.72 377.43 199.87 dis:  0.11 0.13 0.13 0.16
20 5823.18 3897.10 365.89 2.83 dis:  0.12 0.14 0.14 0.16
20 5695.11 2645.04 368.51 32.29 dis:  0.12 0.14 0.14 0.17
40 5687.60 2130.66 376.84 -59.62 dis:  0.17 0.21 0.19 0.23
40 5864.40 1066.36 369.36 -190.44 dis:  0.17 0.21 0.19 0.23
40 5796.03 454.48 366.60 -298.71 dis:  0.19 0.23 0.21 0.25
80 5654.35 483.88 378.30 -341.87 dis:  0.29 0.35 0.30 0.36
80 5677.56 224.40 368.66 -460.74 dis:  0.29 0.36 0.30 0.37
80 5699.11 -15.14 368.10 -762.29 dis:  0.36 0.44 0.38 0.45


In [163]:
def get_action(o, md):
    with torch.no_grad():
        o = torch.as_tensor(o, dtype=torch.float32)
        a = md.act(o)
    return a


def change_env_to_adv(env, md, max_eps_t):
    qpos = deepcopy(env.sim.data.qpos)
    qvel = deepcopy(env.sim.data.qvel)
    grad_pos, grad_vel = get_grad(env, md, max_eps_t)
    grad_pos = np.clip(grad_pos, -1, 1)
    grad_vel = np.clip(grad_vel, -1, 1)
    # adv_qpos = qpos + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nq)
    # adv_qvel = qvel + np.random.uniform(
    #         low=-max_eps_t, high=+max_eps_t, size=env.model.nv)
    adv_qpos = qpos - max_eps_t*grad_pos
    adv_qvel = qvel - max_eps_t*grad_vel
    init_qpos = env.sim.data.qpos.ravel().copy()
    init_qvel = env.sim.data.qvel.ravel().copy()
    # rns = env._reset_noise_scale
    rns = 0.1
    adv_qpos = np.clip(adv_qpos, init_qpos - rns,  init_qpos + rns)
    adv_qvel = np.clip(adv_qvel, init_qvel - 2*rns, init_qvel + 2*rns)
    
    env.set_state(adv_qpos, adv_qvel)
    new_obs = env.get_obs()
    return new_obs

def testing(md, env_name, adv_step, max_eps_t = 0.001):
    test_num = 50
    test_l = 1000
    env = gym.make(env_name)
    orig_rs = []
    adv_rs = []
    orig_qs = []
    adv_qs = []
    for i in range(test_num):
        o = env.reset()
        old_state = save_state(env)
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        orig_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        orig_rs.append(r)
        
        restore_state(env, old_state)
        for _ in range(adv_step):
            o = change_env_to_adv(env, md, max_eps_t)
        
        with torch.torch.no_grad():
            q = get_value(o, md).numpy()
        adv_qs.append(q)
        _, r = run_extra_steps(env, o, 0, md, 1000, test_l)
        adv_rs.append(r)
    print_2f(adv_step, get_mean(orig_rs), get_mean(adv_rs), get_mean(orig_qs), get_mean(adv_qs))
    

testing(m1, 'Ant-v3', 10)
testing(md, 'Ant-v3', 10)
testing(md2, 'Ant-v3', 10)
testing(m1, 'Ant-v3', 20)
testing(md, 'Ant-v3', 20)
testing(md2, 'Ant-v3', 20)
testing(m1, 'Ant-v3', 40)
testing(md, 'Ant-v3', 40)
testing(md2, 'Ant-v3', 40)
testing(m1, 'Ant-v3', 80)
testing(md, 'Ant-v3', 80)
testing(md2, 'Ant-v3', 80)

10 5871.65 5634.48 378.25 373.21
10 5672.27 5641.96 370.84 365.03
10 5829.11 5787.20 365.21 360.05
20 5823.83 5808.04 376.00 365.68
20 5552.02 5774.97 366.45 353.82
20 5715.22 5817.36 368.74 358.97
40 5938.89 5716.21 376.47 354.42
40 5978.07 5540.93 368.98 343.33
40 5730.65 5932.85 368.27 346.69
80 5702.57 5631.14 378.29 330.14
80 5570.50 5729.76 368.67 300.92
80 5682.40 5905.85 367.42 318.93


In [191]:
import random
random.randint(0,3)

3

In [206]:
env.reset()
s = env.sim.get_state()
print(s)
env.sim.forward()
s = env.sim.get_state()
print(s)


MjSimState(time=0.0, qpos=array([-0.02617342, -0.02761698,  0.75605344,  0.99485032, -0.03005331,
        0.03483065,  0.09031313,  0.04466091, -0.03153617,  0.06298635,
        0.06318186, -0.09845077, -0.02295335, -0.02311682, -0.01486309]), qvel=array([ 0.04955178, -0.04639389, -0.0674654 ,  0.03095215, -0.01689994,
        0.1368381 ,  0.07399187, -0.07890817, -0.01606276,  0.1702014 ,
        0.05325084, -0.13882166,  0.03134442, -0.00683104]), act=None, udd_state={})
MjSimState(time=0.0, qpos=array([-0.02617342, -0.02761698,  0.75605344,  0.99485032, -0.03005331,
        0.03483065,  0.09031313,  0.04466091, -0.03153617,  0.06298635,
        0.06318186, -0.09845077, -0.02295335, -0.02311682, -0.01486309]), qvel=array([ 0.04955178, -0.04639389, -0.0674654 ,  0.03095215, -0.01689994,
        0.1368381 ,  0.07399187, -0.07890817, -0.01606276,  0.1702014 ,
        0.05325084, -0.13882166,  0.03134442, -0.00683104]), act=None, udd_state={})


In [197]:
o = env.get_obs()
a = get_action(o, md)
env.step(a)
s2 = env.sim.get_state()
print(s2)


MjSimState(time=23.200000000000827, qpos=array([ 1.34553116e+02, -1.74787395e+01,  5.57658190e-01,  8.90495813e-01,
       -2.39190275e-02, -2.73955727e-02,  4.53535632e-01, -5.26442403e-01,
        5.21601129e-01,  6.27414610e-02, -5.22426270e-01,  5.29494749e-01,
       -5.22962011e-01, -1.48080803e-01,  5.21909865e-01]), qvel=array([ 4.97041445e+00, -1.08096313e+00, -1.60077800e+00,  2.88261376e-01,
       -8.11529663e-01, -3.22964140e-01,  6.00473944e-02, -5.30573276e-03,
        8.61205332e+00,  5.24909571e-03, -1.42019404e-01, -2.34475657e-02,
        7.51371280e-01, -4.53819375e-04]), act=None, udd_state={})


In [198]:
env.sim.set_state(s)
env.sim.forward()
s = env.sim.get_state()
print(s)

MjSimState(time=23.15000000000082, qpos=array([ 1.34300706e+02, -1.74218079e+01,  6.25926612e-01,  8.95516314e-01,
       -3.69560323e-02, -1.06297132e-02,  4.43364175e-01, -5.35470623e-01,
        5.21944544e-01, -2.29842399e-01, -5.22659800e-01,  5.50305118e-01,
       -5.20308676e-01, -8.98953268e-02,  5.21159316e-01]), qvel=array([ 5.10619435e+00, -1.20159320e+00, -1.12681048e+00,  5.72246202e-02,
       -8.59304756e-01,  1.19870253e+00,  3.83812317e-01,  2.02744936e-03,
        3.04500041e+00, -1.12547319e-02, -8.65269143e-01, -8.01163914e-02,
       -3.11102177e+00,  6.19502554e-02]), act=None, udd_state={})


In [200]:
l = [1,2,3]
l2 = [1,2,3]
l3 = l+l2
print(l3)

[1, 2, 3, 1, 2, 3]


In [202]:
l3.sort()
print(l3)

[1, 1, 2, 2, 3, 3]


In [215]:
print(torch.__version__)
md = torch.load('./ppo-ant.pt')


1.3.1


In [217]:
md.keys()

dict_keys(['pi', 'obs_mean', 'obs_std', 'clip'])

In [218]:
import torch.nn as nn
def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)
class Actor():
    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        self.pi = mlp([obs_dim] + list(hidden_sizes) + [act_dim], activation)
        self.obs_mean = np.ones(obs_dim)
        self.obs_std = np.ones(obs_dim)
        self.clip = 10.0
        print(type(self.pi))
    
    def normalize_o(self, o):
        o = o - self.obs_mean
        o = o / (self.obs_std + 1e-8)
        o = np.clip(o, -self.clip, self.clip)
        return o
    
    def act(self, o):
        if torch.is_tensor(o):
            o = o.numpy()
        o = self.normalize_o(o)
        o = torch.as_tensor(o, dtype=torch.float32)
        return self.pi(o).detach().numpy()
    
    def copy_model(self, md):
        self.pi.load_state_dict(md['pi'])
        self.obs_mean = md['obs_mean']
        self.obs_std = md['obs_std']
        self.clip = md['clip']

x = Actor(111, 8, (64, 64), nn.Tanh)
md = torch.load('./ppo-ant.pt')
x.copy_model(md)

<class 'torch.nn.modules.container.Sequential'>


In [221]:
import gym
from copy import deepcopy 
env = gym.make('Ant-v3')

def get_action(o, md):
    o = torch.as_tensor(o, dtype=torch.float32)
    return md.act(o)

def test_model(env, model, max_ep_len=1000, num_episodes=5):
    o, r, d, ep_ret, ep_len, n = env.reset(), 0, False, 0, 0, 0
    total_rewards = []
    trajs = []
    mid_points = []
    while n < num_episodes:
        old_state = deepcopy(env.sim.get_state())
        mid_points.append((ep_len, old_state, o))
        a = get_action(o, model)
        o, r, d, _ = env.step(a)
        ep_ret += r
        ep_len += 1
        if d or (ep_len == max_ep_len):
            total_rewards.append(ep_ret)
            trajs.append(mid_points)
            mid_points = [] # clear mid point
            o, r, d, ep_ret, ep_len = env.reset(), 0, False, 0, 0
            n += 1
    return total_rewards, trajs

rs ,tjs = test_model(env, x)

def vmean(v):
    return sum(v) / len(v)

print(vmean(rs))

5857.734883010637


In [223]:
env = gym.make('Ant-v3')
print(env.observation_space.shape[0])
print(env.action_space.shape[0])

111
8


In [227]:
def print_2f_tab(*args):
    __builtins__.print(*("%.2f" % a if isinstance(a, float) else a
                         for a in args), sep='\t')

a = [1.233543, 23.12313412, 2321.23]
print_2f_tab(*a)

1.23	23.12	2321.23
